In [2]:
from bs4 import BeautifulSoup
from IPython.core.display import display, HTML
import sqlite3
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

In [34]:
pd.set_option('display.max_colwidth', -1)
plt.rcdefaults()

In [3]:
conn = sqlite3.connect('../Data/crossvalidated.db')

In [6]:
# return all the records for questions posts from posts table
ques_query = "SELECT * FROM [posts]"

In [7]:
post_df = pd.read_sql_query(ques_query, conn)

In [8]:
post_df.shape

(151044, 21)

In [9]:
post_df.drop(['LastEditorDisplayName','CommunityOwnedDate','LastEditorUserId','LastEditDate',
             'LastActivityDate'],axis=1,inplace=True)

In [11]:
post_df.columns

Index([u'Body', u'ViewCount', u'ClosedDate', u'ParentID', u'CommentCount',
       u'AnswerCount', u'AcceptedAnswerId', u'Score', u'OwnerDisplayName',
       u'Title', u'PostTypeId', u'OwnerUserId', u'Tags', u'CreationDate',
       u'FavoriteCount', u'Id'],
      dtype='object')

#Helper functions

In [12]:
def makeplain(html):
    text = BeautifulSoup(html, 'html.parser').get_text()
    text = text.replace(u'\u200b','')
    text = text.replace(u'\xa0','')
    return text

In [13]:
post_df['Body_Text'] = post_df.apply(lambda row: makeplain(row['Body']), axis=1)

In [23]:
test = post_df.head(4)

#Top Questions

In [28]:
top_100ques = post_df[(post_df.PostTypeId==1)].sort(['Score','ViewCount','AnswerCount','FavoriteCount','CommentCount'],\
                                      ascending=[False,False,False,False,False]).head(100)

/usr/local/lib/python2.7/site-packages/IPython/kernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [35]:
top_100ques[['Score','Body_Text','Title','ViewCount','AnswerCount','FavoriteCount','CommentCount']].head()

,Score,Body_Text,Title,ViewCount,AnswerCount,FavoriteCount,CommentCount
2504,334,"In today's pattern recognition class my professor talked about PCA, eigenvectors & eigenvalues. \nI got the mathematics of it. If I'm asked to find eigenvalues etc. I'll do it correctly like a machine. But I didn't understand it. I didn't get the purpose of it. I didn't get the feel of it. I strongly believe in \n\nyou do not really understand something unless you can explain it to your grandmother -- Albert Einstein\n\nWell, I can't explain these concepts to a layman or grandma.\n\nWhy PCA, eigenvectors & eigenvalues? What was the need for these concepts?\nHow would you explain these to a layman?\n\n","Making sense of principal component analysis, eigenvectors & eigenvalues",134213.0,26.0,389.0,15
1489,272,"Lots of people use a main tool like Excel or another spreadsheet, SPSS, Stata, or R for their statistics needs. They might turn to some specific package for very special needs, but a lot of things can be done with a simple spreadsheet or a general stats package or stats programming environment.\nI've always liked Python as a programming language, and for simple needs, it's easy to write a short program that calculates what I need. Matplotlib allows me to plot it.\nHas anyone switched completely from, say R, to Python? R (or any other statistics package) has a lot of functionality specific to statistics, and it has data structures that allow you to think about the statistics you want to perform and less about the internal representation of your data. Python (or some other dynamic language) has the benefit of allowing me to program in a familiar, high-level language, and it lets me programmatically interact with real-world systems in which the data resides or from which I can take measurements. But I haven't found any Python package that would allow me to express things with ""statistical terminology"" – from simple descriptive statistics to more complicated multivariate methods.\nWhat can you recommend if I wanted to use Python as a ""statistics workbench"" to replace R, SPSS, etc.?\nWhat would I gain and lose, based on your experience?\n",Python as a statistics workbench,95622.0,25.0,328.0,1
5,226,"Last year, I read a blog post from Brendan O'Connor entitled ""Statistics vs. Machine Learning, fight!"" that discussed some of the differences between the two fields. Andrew Gelman responded favorably to this:\nSimon Blomberg: \n\nFrom R's fortunes\n package: To paraphrase provocatively,\n 'machine learning is statistics minus\n any checking of models and\n assumptions'.\n -- Brian D. Ripley (about the difference between machine learning\n and statistics) useR! 2004, Vienna\n (May 2004) :-) Season's Greetings!\n\nAndrew Gelman:\n\nIn that case, maybe we should get rid\n of checking of models and assumptions\n more often. Then maybe we'd be able to\n solve some of the problems that the\n machine learning people can solve but\n we can't!\n\nThere was also the ""Statistical Modeling: The Two Cultures"" paper by Leo Breiman in 2001 which argued that statisticians rely too heavily on data modeling, and that machine learning techniques are making progress by instead relying on the predictive accuracy of models.\nHas the statistics field changed over the last decade in response to these critiques? Do the two cultures still exist or has statistics grown to embrace machine learning techniques such as neural networks and support vector machines?\n",The Two Cultures: statistics vs. machine learning?,65657.0,17.0,211.0,6
398,217,This is one of my favorites:\n\nOne entry per answer. This is in the vein of the Stack Overflow question What’s your favorite “programmer” cartoon?.\nP.S. Do not hotlink the cartoon without the site's permission please.\n,"What is your favorite ""data analysis"" cartoon?",103708.0,68.0,196.0,10
108,208,"In the definition of standard deviation, why do we have to square the difference from the mean to get the mean (E) and take the squar

In [36]:
best_ques = top_100ques.Title.values

In [37]:
for ques in best_ques[:20]:
    display(HTML(ques))

#Top Answers

In [38]:
top_100ans = post_df[(post_df.PostTypeId==2)].sort(['Score'],ascending=[False]).head(100)

/usr/local/lib/python2.7/site-packages/IPython/kernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [39]:
top_100ans[['Score','Body_Text','Tags']].head()

Score  \
99888  259     
27635  251     
1525   231     
39438  225     
2512   220     

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [42]:
best_ans = top_100ans.Body.values
i=1
for ans in best_ans[:5]:
    print 'Top '+str(i)+ ' Answer'
    display(HTML(ans))
    print '------------------------------------------------------'
    i+=1   

Top 1 Answer


------------------------------------------------------
Top 2 Answer


------------------------------------------------------
Top 3 Answer


------------------------------------------------------
Top 4 Answer


------------------------------------------------------
Top 5 Answer


------------------------------------------------------


#Who's really contributing to the community?